In [ ]:
from onceorauto.image.barrel_correct import auto_undistort, map_image
import tensorflow as tf
from tensorpack import Conv2D


image = cv2.imread('./4k.jpg')
with open('./4k.json', 'r') as f:
    data = json.load(f)
pt = data['shapes'][0]['points']
pts = [np.array(pt)]

image_t, c, r = auto_undistort(image, pts)


map_x = r[:, 0].reshape((2160, 3840)).astype(np.half)
map_y = r[:, 1].reshape((2160, 3840)).astype(np.half)
map_yx = np.stack([map_y, map_x], axis=-1) # opencv 的 map
# opencv map -> tf flow
cor_xy = np.indices((2160, 3840)).transpose(1, 2, 0)
flow = cor_xy - map_yx

In [ ]:
inputs = tf.placeholder(tf.half, shape=[1, 2160, 3840, 3], name='image')
inputs = Conv2D('conv2d1', inputs, filters=256, kernel_size=3)
inputs = Conv2D('conv2d2', inputs, filters=3, kernel_size=3)
flow_tf = tf.placeholder(tf.half, shape=[1, None, None, 2], name='flow')
image_t_tf = tf.contrib.image.dense_image_warp(inputs, flow_tf)
init = tf.global_variables_initializer()

In [ ]:
sess = tf.Session()
sess.run(init)

In [ ]:
flow = np.expand_dims(flow, 0)
image = np.expand_dims(image, 0)

In [ ]:
image = tf.data.experimental.prefetch_to_device()

In [ ]:
%%timeit
out = tf.get_default_graph().get_tensor_by_name('conv2d2/output:0')
outputs = sess.run(out, feed_dict={"image:0":image, "flow:0":flow})

In [ ]:
%%timeit
outputs = sess.run(image_t_tf, feed_dict={"image:0":image, "flow:0":flow})

In [ ]:
plt.imshow(outputs[0].astype(np.uint8))